# Load garnet-bearing experiments from LEPR

## Initialize required packages

In [ ]:
import thermoengine as thermo
%run "core.ipynb"
%load_ext snakeviz

## Load experimental data
### loads standard LEPR data from excel file, converts to pandas df and saves components of excel file into separate objects

    * input:

        filename: str
            name of excel file

        data_dir: keyword arg; data_dir = str
            directory location of excel file

    * output:
        
        exp_data: OrderedDict
            saves specific columns of "Experiment" tab into ordered dictionary with 
            column names as keys
        
        metadata: OrderedDict
            saves all columns regarding experimental conditions from "Experiment" tab
            
        ph_data: OrderedDict
            saves all phase tabs
        
        phs_symbols: OrderedDict
            stores phase_symbols tab from LEPR file
 


In [ ]:
exp_data, metadata, phs_data, phs_symbols = load_LEPR_data('grt_bearing_expts.xls', data_dir='data/')

In [ ]:
exp_data.keys()

In [ ]:
metadata.keys()

In [ ]:
phs_data

In [ ]:
phs_data.keys()

In [ ]:
phs_symbols['phase_symbol']

### Change key names in each phase tab to cleaner format

In [ ]:
phs_wt_comp = major_wt_oxide_LEPR_data(phs_data)
phs_wt_comp

### Convert from wt% oxide to mol oxide compositions for all phase compositions

In [ ]:
phs_mol_comp = OrderedDict()
for phs_tab_name in phs_wt_comp:
    phs_tab = phs_wt_comp[phs_tab_name]
    
    mol_oxide = thermo.chem.wt_to_mol_oxide(phs_tab.values)
    phs_mol_comp[phs_tab_name] = pd.DataFrame(
        mol_oxide,  index=phs_tab.index, columns=phs_tab.columns)

In [ ]:
phs_mol_comp['Liquid'].head()

In [ ]:
analysis={}
analysis['exp_data'] = exp_data
analysis['phs_data'] = phs_data
analysis['phs_symbols'] = phs_symbols
analysis['phs_mol_comp'] = phs_mol_comp
analysis['phs_wt_comp'] = phs_wt_comp

save_analysis(analysis)

# LEPR data visualization

### Phases present in experiments

In [ ]:
print('List of experimental phases:', *phs_data.keys(), sep='\n- ')

### Number of experiments 

In [ ]:
number_of_experiments = len(exp_data)
print('The number of experiments =', number_of_experiments)

### Number of unique phase assemblages in the experiments

In [ ]:
unique_phs_assemblages = len(metadata['Phases'].unique())
print('The number of unique phase assemblages =', unique_phs_assemblages)

### Starting material bulk composition space covered by experiments

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
silica = exp_data['SiO2']
total_alkalis =exp_data['Na2O'] + exp_data['K2O']

plt.figure(figsize=(8,5))
ax1 = plt.subplot(111)
add_LeMaitre_fields(ax1)
plt.plot(silica, total_alkalis, 'ro')
plt.xlabel('SiO2 (wt%)')
plt.ylabel('Na2O + K2O (wt%)')
plt.show()

### Pressure-temperature space of experiments

In [ ]:
temperatures = exp_data['T']
pressures = exp_data['P']

plt.figure()
plt.plot(temperatures, pressures, 'bo')
plt.xlabel('Temperature (C)')
plt.ylabel('Pressure (GPa)')
plt.title('LEPR data P-T space')
plt.show()

In [ ]:
plt.figure()
plt.hist(pressures, bins='auto')  # arguments are passed to np.histogram
plt.title("Range of experimental pressures")
plt.xlabel('Pressure (GPa)')
plt.show()